## Assignment 1
Simple Data Encryption Standard (S-DES)

[Reference](https://www.youtube.com/watch?v=mS2VC7vbbNc&t=543s)

In [1]:
import random

### Constants initialisation

In [2]:
P10 = [] # Random permutation of size 10
P8 = [] # Sample 8 elemets from P10
EP = [] # Shuffle two permutations of size 4
IP = [] # # Random permutation of size 8
IP_INV = [] # To be calculated from IP IP_INV[IP[i] - 1] = i + 1

In [3]:
def gen_random_permutation(n):
  out = [i+1 for i in range(n)]
  random.shuffle(out)
  return out;

In [4]:
IP = gen_random_permutation(8)
IP_INV = [1]*8
for i in range(len(IP)):
  IP_INV[IP[i]-1] = i+1
print("IP:", IP)
print("IP_INV:", IP_INV)

IP: [4, 3, 8, 7, 6, 5, 1, 2]
IP_INV: [7, 8, 2, 1, 6, 5, 4, 3]


In [5]:
EP = gen_random_permutation(4)*2
print("EP:", EP)

EP: [3, 1, 4, 2, 3, 1, 4, 2]


In [6]:
P10 = gen_random_permutation(10)
print("P10:", P10)

P10: [9, 6, 5, 10, 7, 8, 3, 2, 1, 4]


In [7]:
P8 = random.sample(gen_random_permutation(10), 8)
print("P8:", P8)

P8: [8, 2, 7, 3, 4, 9, 1, 10]


In [8]:
S0 = [
    [1, 0, 3, 2],
    [3, 2, 1, 0],
    [0, 2, 1, 3],
    [3, 1, 3, 2]
]
S1 = [
    [0, 1, 2, 3],
    [2, 0, 1, 3],
    [3, 0, 1, 0],
    [2, 1, 0, 3]
]

### Helper functions

In [9]:
def bin_to_dec(x):
  return int(x, 2)
def dec_to_bin(x):
  return bin(x).replace("0b","")

In [10]:
def left_circular_shift(x, shifts=1):
  shifts = shifts % len(x)
  return x[shifts:] + x[:shifts]

In [11]:
def permutate(key, perm):
  ret = ""
  for k in perm:
    ret += key[k-1]
  return ret

In [12]:
def split_str(key):
  half = len(key)//2
  key1 = key[:half]
  key2 = key[half:]
  return key1, key2

In [13]:
def xor(a, b):
  ret = ""
  for i in range(len(a)):
    if a[i] == b[i]: ret += "0"
    else: ret += "1"
  return ret

### Algorithm necessary functions

In [14]:
def gen_subkeys(key):
  n_key = permutate(key, P10)

  left_key, right_key = split_str(n_key)

  left_key = left_circular_shift(left_key, 1)
  right_key = left_circular_shift(right_key, 1)

  k1 = permutate(left_key + right_key, P8)

  left_key = left_circular_shift(left_key, 2)
  right_key = left_circular_shift(right_key, 2)

  k2 = permutate(left_key + right_key, P8)

  return k1, k2

In [15]:
def s_box(text, s):
  r = text[0] + text[3]
  c = text[1] + text[2]

  r = bin_to_dec(r)
  c = bin_to_dec(c)
  out = s[r][c]
  out = dec_to_bin(out)
  while len(out) < 2:
    out = "0" + out
  return out

In [16]:
def function(left, right, subkey):
  text = right
  text = permutate(text, EP)
  text = xor(text, subkey)
  text_left, text_right = split_str(text)
  text = s_box(text_left, S0) + s_box(text_right, S1)
  text = xor(text, left)
  return text, right

In [17]:
def encryption(plaintext, key):
  k1, k2 = gen_subkeys(key)

  ciphertext = permutate(plaintext, IP)

  left, right = split_str(ciphertext)
  left, right = function(left, right, k1)

  left, right = right, left

  left, right = function(left, right, k2)

  ciphertext = permutate(left + right, IP_INV)

  return ciphertext

In [18]:
def decryption(ciphertext, key):
  k1, k2 = gen_subkeys(key)

  plaintext = permutate(ciphertext, IP)

  left, right = split_str(plaintext)
  left, right = function(left, right, k2)

  left, right = right, left

  left, right = function(left, right, k1)

  plaintext = permutate(left + right, IP_INV)

  return plaintext

### Testing

In [19]:
key = "1010001011"
plaintext = "10001010"

In [20]:
c = encryption(plaintext, key)
p = decryption(c, key)

In [21]:
assert(p==plaintext)